In [1]:
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import base64
import io
from PIL import Image

In [ ]:
def get_cactus_height(x,y, img):
    while np.any(img[x,y-7:y+7]):
        x -= 1
    return x

def find_cacti(img):
    height,width = np.shape(img)
    dino = img[height//2:,:height//2]
    vision = round(height*.8)
    bird_vision = round(height*0.6)
    field = img[vision,height//2 - width//30 - 5:] + img[bird_vision,height//2 - width//30 - 5:]
    distance = []
    cacti_height = []
    prev_pixel = field[0]
    cur_dist = 0
    for i,pixel in enumerate(field):
        if prev_pixel == 0 and pixel != 0:
            distance.append(i)
            cacti_height.append(get_cactus_height(vision, height//2 - width//30 - 5 + i, img))
        prev_pixel = pixel
    return distance, cacti_height

In [ ]:
js_call = "return document.getElementsByClassName('runner-canvas')[0].toDataURL('image/png').substring(21);"
is_jumping_js = 'return Runner.instance_.tRex.jumping'
second_js = "Runner.instance_.tRex.startJump({})"
end_js = "Runner.instance_.tRex.endJump({})"
cur_speed_js = "return Runner.instance_.currentSpeed"
cur_dist_js = "return Runner.instance_.distanceRan"
url = "https://chromedino.com/"
browser = webdriver.Chrome()
try:
    browser.get(url)

    # start the first game of dino run
    body = browser.find_element_by_id('t')
    body.send_keys(Keys.ARROW_UP)
    
    prev_dist = 0

    speed_factor = 0.999
    counter = 0
    while True:
        time.sleep(.0015)
        
        speed_factor += .00025
        print('speedf', speed_factor)
        if speed_factor > 2.95:
            speed_factor = 2.95
        b64_image = browser.execute_script(js_call)
        image = base64.b64decode(b64_image)
        buf = io.BytesIO(image)
        img = Image.open(buf).convert('LA')
        np_img = np.array(img)[:,:,0]
        distances, heights = find_cacti(np_img)
        #print(distances, heights)
        counter += 1
        if counter % 60 == 0:
            cur_dista = browser.execute_script(cur_dist_js)
            print(prev_dist, cur_dista)
            if prev_dist == cur_dista and cur_dista > 20:
                speed_factor = 0.999
                command = input("Type Q to quit")
                if command == 'Q':
                    break
                else:
                    body.send_keys(Keys.ARROW_UP)
                # This is a game over
            else:
                prev_dist = cur_dista

        if len(distances) != 0:
            distances_copy = distances.copy()
            for i,v in enumerate(distances_copy):
                try:
                    if abs(distances_copy[i+1] - v) < 25:
                        distances.remove(v)
                    else:
                        break
                except:
                    pass
            print(distances, distances_copy)
            # We know we need to jump over something, how tall is it?
            #if browser.execute_script(is_jumping_js):
                #print('jumping')
                #if not np.any(np_img[20:80, 120]):
                    #print('end jump')
                    #if distances[0] > 30*speed_factor:
                        #cur_speed = float(browser.execute_script(cur_speed_js))
                        #browser.execute_script(end_js.format(cur_speed))
            if 85 <= heights[0] <= 94:
                if distances[0] < (98 * speed_factor):
                    cur_speed = float(browser.execute_script(cur_speed_js))
                    browser.execute_script(second_js.format(cur_speed))
            elif 95 <= heights[0] <= 103:
                if distances[0] < (105 * speed_factor):
                    cur_speed = float(browser.execute_script(cur_speed_js))
                    browser.execute_script(second_js.format(cur_speed))
            elif 104 <= heights[0]:
                if distances[0] < (105 * speed_factor):
                    cur_speed = float(browser.execute_script(cur_speed_js))
                    browser.execute_script(second_js.format(cur_speed))
        
except Exception as e:
    print("Errored out at step 1", e)
finally:
    input("Enter any Key to Close the browser")
    browser.close() #make sure we close it when we are finished.

speedf 0.99925
speedf 0.9994999999999999
speedf 0.9997499999999999
speedf 0.9999999999999999
speedf 1.0002499999999999
speedf 1.0005
speedf 1.00075
speedf 1.0010000000000001
speedf 1.0012500000000002
speedf 1.0015000000000003
speedf 1.0017500000000004
speedf 1.0020000000000004
speedf 1.0022500000000005
speedf 1.0025000000000006
speedf 1.0027500000000007
speedf 1.0030000000000008
speedf 1.0032500000000009
speedf 1.003500000000001
speedf 1.003750000000001
speedf 1.0040000000000011
speedf 1.0042500000000012
speedf 1.0045000000000013
speedf 1.0047500000000014
speedf 1.0050000000000014
speedf 1.0052500000000015
speedf 1.0055000000000016
speedf 1.0057500000000017
speedf 1.0060000000000018
speedf 1.0062500000000019
speedf 1.006500000000002
speedf 1.006750000000002
speedf 1.0070000000000021
speedf 1.0072500000000022
speedf 1.0075000000000023
speedf 1.0077500000000024
speedf 1.0080000000000024
speedf 1.0082500000000025
speedf 1.0085000000000026
speedf 1.0087500000000027
speedf 1.009000000000002

[214, 505] [214, 505]
speedf 1.0735000000000243
[208, 499] [208, 499]
speedf 1.0737500000000244
[202, 493] [202, 493]
speedf 1.0740000000000245
1250.9908200000082 1619.9498400000123
[196, 487] [196, 487]
speedf 1.0742500000000246
[190, 481] [190, 481]
speedf 1.0745000000000247
[184, 475] [184, 475]
speedf 1.0747500000000247
[178, 469] [178, 469]
speedf 1.0750000000000248
[172, 463] [172, 463]
speedf 1.075250000000025
[166, 457] [166, 457]
speedf 1.075500000000025
[160, 451] [160, 451]
speedf 1.075750000000025
[154, 445] [154, 445]
speedf 1.0760000000000252
[148, 439] [148, 439]
speedf 1.0762500000000252
[142, 433] [142, 433]
speedf 1.0765000000000253
[136, 427] [136, 427]
speedf 1.0767500000000254
[130, 421] [130, 421]
speedf 1.0770000000000255
[123, 414] [123, 414]
speedf 1.0772500000000256
[118, 409] [118, 409]
speedf 1.0775000000000257
[112, 403] [112, 403]
speedf 1.0777500000000257
[107, 398] [107, 398]
speedf 1.0780000000000258
[101, 392] [101, 392]
speedf 1.078250000000026
[89, 3

speedf 1.1197500000000398
[90, 382] [72, 90, 382]
speedf 1.1200000000000399
[78, 370] [60, 78, 370]
speedf 1.12025000000004
[65, 357] [47, 65, 357]
speedf 1.12050000000004
[59, 351] [41, 59, 351]
speedf 1.12075000000004
[47, 339] [29, 47, 339]
speedf 1.1210000000000402
[41, 333] [23, 41, 333]
speedf 1.1212500000000403
[28, 320] [10, 28, 320]
speedf 1.1215000000000404
[22, 314] [4, 22, 314]
speedf 1.1217500000000404
[16, 308] [16, 308]
speedf 1.1220000000000405
[4, 296] [4, 296]
speedf 1.1222500000000406
[284, 527] [284, 527]
speedf 1.1225000000000407
[278, 521] [278, 521]
speedf 1.1227500000000408
[272, 515] [272, 515]
speedf 1.1230000000000409
[266, 509] [266, 509]
speedf 1.123250000000041
[260, 503] [260, 503]
speedf 1.123500000000041
[260, 503] [260, 503]
speedf 1.123750000000041
[253, 496] [253, 496]
speedf 1.1240000000000412
[247, 490] [247, 490]
speedf 1.1242500000000413
[242, 485] [242, 485]
speedf 1.1245000000000414
[236, 479] [236, 479]
speedf 1.1247500000000414
[230, 473] [23

[222, 496] [222, 496]
speedf 1.162250000000054
[211, 485] [211, 485]
speedf 1.162500000000054
[203, 477] [203, 477]
speedf 1.1627500000000541
[192, 466] [192, 466]
speedf 1.1630000000000542
[184, 458] [184, 458]
speedf 1.1632500000000543
[178, 452] [178, 452]
speedf 1.1635000000000544
[172, 446] [172, 446]
speedf 1.1637500000000545
[166, 440] [166, 440]
speedf 1.1640000000000545
3838.079640000065 4378.356840000082
[160, 434] [160, 434]
speedf 1.1642500000000546
[149, 423] [149, 423]
speedf 1.1645000000000547
[141, 415] [141, 415]
speedf 1.1647500000000548
[135, 409] [135, 409]
speedf 1.1650000000000549
[124, 398] [124, 398]
speedf 1.165250000000055
[116, 390] [116, 390]
speedf 1.165500000000055
[110, 384] [110, 384]
speedf 1.1657500000000551
[99, 373] [99, 373]
speedf 1.1660000000000552
[86, 360] [86, 360]
speedf 1.1662500000000553
[73, 347] [73, 347]
speedf 1.1665000000000554
[60, 334] [60, 334]
speedf 1.1667500000000555
[48, 322] [48, 322]
speedf 1.1670000000000555
[36, 310] [36, 310

speedf 1.2020000000000672
[128, 488] [110, 128, 488]
speedf 1.2022500000000673
[117, 477] [99, 117, 477]
speedf 1.2025000000000674
[104, 464] [86, 104, 464]
speedf 1.2027500000000675
[91, 451] [73, 91, 451]
speedf 1.2030000000000676
[78, 438] [60, 78, 438]
speedf 1.2032500000000677
[65, 425] [47, 65, 425]
speedf 1.2035000000000677
[52, 412] [34, 52, 412]
speedf 1.2037500000000678
[39, 399] [21, 39, 399]
speedf 1.204000000000068
[26, 386] [8, 26, 386]
speedf 1.204250000000068
[13, 373] [13, 373]
speedf 1.204500000000068
[360] [360]
speedf 1.2047500000000682
[352] [352]
speedf 1.2050000000000682
[345] [345]
speedf 1.2052500000000683
[338] [338]
speedf 1.2055000000000684
[332] [332]
speedf 1.2057500000000685
[325] [325]
speedf 1.2060000000000686
[313] [313]
speedf 1.2062500000000687
[305] [305]
speedf 1.2065000000000687
[299] [299]
speedf 1.2067500000000688
[292] [292]
speedf 1.207000000000069
[285] [285]
speedf 1.207250000000069
[279] [279]
speedf 1.207500000000069
[272] [272]
speedf 1.2

[506] [506]
speedf 1.243500000000081
[494] [494]
speedf 1.2437500000000812
[481] [481]
speedf 1.2440000000000813
[473] [473]
speedf 1.2442500000000813
[466] [466]
speedf 1.2445000000000814
[459] [459]
speedf 1.2447500000000815
[453] [453]
speedf 1.2450000000000816
[446] [446]
speedf 1.2452500000000817
[439] [439]
speedf 1.2455000000000818
[433] [433]
speedf 1.2457500000000818
[426] [426]
speedf 1.246000000000082
[419] [419]
speedf 1.246250000000082
[412] [412]
speedf 1.246500000000082
[400] [400]
speedf 1.2467500000000822
[392] [392]
speedf 1.2470000000000823
[385] [385]
speedf 1.2472500000000823
[378] [378]
speedf 1.2475000000000824
[371] [371]
speedf 1.2477500000000825
[365] [365]
speedf 1.2480000000000826
[358] [358]
speedf 1.2482500000000827
[352] [352]
speedf 1.2485000000000828
[345] [345]
speedf 1.2487500000000828
[338] [338]
speedf 1.249000000000083
[326] [326]
speedf 1.249250000000083
[317] [317]
speedf 1.249500000000083
[311] [311]
speedf 1.2497500000000832
[305, 527] [305, 52

speedf 1.2892500000000964
[303, 334] [303, 334]
speedf 1.2895000000000965
[290, 321] [290, 321]
speedf 1.2897500000000965
[282, 313] [282, 313]
speedf 1.2900000000000966
[275, 306] [275, 306]
speedf 1.2902500000000967
[268, 299] [268, 299]
speedf 1.2905000000000968
[261, 292] [261, 292]
speedf 1.2907500000000969
[254, 285] [254, 285]
speedf 1.291000000000097
[247, 278] [247, 278]
speedf 1.291250000000097
[240, 271] [240, 271]
speedf 1.2915000000000971
[233, 264] [233, 264]
speedf 1.2917500000000972
[226, 257] [226, 257]
speedf 1.2920000000000973
[214, 245] [214, 245]
speedf 1.2922500000000974
[200, 231] [200, 231]
speedf 1.2925000000000975
[187, 218] [187, 218]
speedf 1.2927500000000975
[179, 210] [179, 210]
speedf 1.2930000000000976
[172, 203] [172, 203]
speedf 1.2932500000000977
[165, 196] [165, 196]
speedf 1.2935000000000978
[158, 189] [158, 189]
speedf 1.2937500000000979
[151, 182] [151, 182]
speedf 1.294000000000098
[144, 175] [144, 175]
speedf 1.294250000000098
[132, 163] [132, 1

[332] [332]
speedf 1.336000000000112
[325] [325]
speedf 1.336250000000112
[318] [318]
speedf 1.3365000000001122
[306] [306]
speedf 1.3367500000001122
[297] [297]
speedf 1.3370000000001123
[290] [290]
speedf 1.3372500000001124
[283] [283]
speedf 1.3375000000001125
[276] [276]
speedf 1.3377500000001126
[269] [269]
speedf 1.3380000000001127
[262] [262]
speedf 1.3382500000001127
[255, 527] [255, 527]
speedf 1.3385000000001128
[248, 520] [248, 520]
speedf 1.338750000000113
[241, 513] [241, 513]
speedf 1.339000000000113
[228, 500] [228, 500]
speedf 1.339250000000113
[219, 491] [219, 491]
speedf 1.3395000000001132
[212, 484] [212, 484]
speedf 1.3397500000001132
[205, 477] [205, 477]
speedf 1.3400000000001133
[198, 470] [198, 470]
speedf 1.3402500000001134
[191, 463] [191, 463]
speedf 1.3405000000001135
[184, 456] [184, 456]
speedf 1.3407500000001136
[177, 449] [177, 449]
speedf 1.3410000000001137
[170, 442] [170, 442]
speedf 1.3412500000001137
[163, 435] [163, 435]
speedf 1.3415000000001138
[

speedf 1.3845000000001282
[103, 390] [103, 390]
speedf 1.3847500000001283
[87, 374] [87, 374]
speedf 1.3850000000001284
[72, 359] [72, 359]
speedf 1.3852500000001284
[57, 344] [57, 344]
speedf 1.3855000000001285
[42, 329] [42, 329]
speedf 1.3857500000001286
[27, 314] [27, 314]
speedf 1.3860000000001287
[12, 299] [12, 299]
speedf 1.3862500000001288
[283] [283]
speedf 1.3865000000001289
[268] [268]
speedf 1.386750000000129
[259, 502, 533] [259, 502, 533]
speedf 1.387000000000129
[251, 494, 525] [251, 494, 525]
speedf 1.387250000000129
[243, 486, 517] [243, 486, 517]
speedf 1.3875000000001292
[236, 479, 510] [236, 479, 510]
speedf 1.3877500000001293
[228, 471, 502] [228, 471, 502]
speedf 1.3880000000001294
[220, 463, 494] [220, 463, 494]
speedf 1.3882500000001294
[213, 456, 487] [213, 456, 487]
speedf 1.3885000000001295
[205, 448, 479] [205, 448, 479]
speedf 1.3887500000001296
[197, 440, 471] [197, 440, 471]
speedf 1.3890000000001297
13284.038700000605 13952.387400000664
[190, 433, 464] [